<a href="https://colab.research.google.com/github/seeedata/BADA/blob/main/Project/1012%20Amazon%20Review%20Crawling%20HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
#페이지 넘기기

urls = []
for num in range(1,54):
    url = 'https://www.amazon.com/-/ko/product-reviews/B007LKKULG/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=' + str(num)
    urls.append(url)

print(len(urls))
urls

53


['https://www.amazon.com/-/ko/product-reviews/B007LKKULG/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=1',
 'https://www.amazon.com/-/ko/product-reviews/B007LKKULG/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=2',
 'https://www.amazon.com/-/ko/product-reviews/B007LKKULG/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=3',
 'https://www.amazon.com/-/ko/product-reviews/B007LKKULG/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=4',
 'https://www.amazon.com/-/ko/product-reviews/B007LKKULG/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=5',
 'https://www.amazon.com/-/ko/product-reviews/B007LKKULG/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=6',
 'https://www.amazon.com/-/ko/product-reviews/B007LKKULG/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=7',
 'https://www.amazon

In [ ]:
#크롤링

driver = webdriver.Chrome()

Reviewers = []
Scores = []
Review = []

for u in urls:
    driver.get(u)
    time.sleep(3)

    #현재 브라우저의 페이지 소스 코드를 html 파서로 파싱
    doc = BeautifulSoup(driver.page_source, 'html')

    #'상위 긍정적, 비판적 리뷰' 영역 제거
    doc.find(class_ = 'a-section a-spacing-large celwidget').decompose()
    #팝업창 제거
    for n in doc.find_all(class_ = 'a-popover-preload'):  #팝업창
        n.decompose()


    #리뷰어 크롤링
    #리뷰어 이름 개수 세기
    length = len(doc.find_all(class_ = 'a-profile-name'))
    #리뷰어 이름 text로 추출해서 name에 저장
    name = [doc.find_all(class_ = 'a-profile-name')[n].text for n in range(0,length)]
    for n in name:
        Reviewers.append(n)


    #평점 크롤링
    rating = [doc.find_all(class_ = 'a-icon-alt')[n].text.strip() for n in range(0,length)]
    for r in rating:
        score = float(r[-3:-1])
        Scores.append(score)


    #리뷰 내용 크롤링
    review_body = [doc.find_all(class_ = 'a-row a-spacing-small review-data')[n].text.strip() for n in range(0,length)]
    for rb in review_body:
        Review.append(rb)

    time.sleep(5)

In [ ]:
driver.quit()

In [ ]:
import pandas as pd

In [ ]:
Review_Table = pd.DataFrame({'Reviewers' : Reviewers, 'Score' : Scores, 'Review' : Review})
Review_Table

,Reviewers,Score,Review
0,JOANIE,4.0,I am glad to have this nice perfume however I ...
1,Elaine,4.0,Chanel 5 Eau de toilette light and fresh fragr...
2,Daniela Guzman.C,5.0,Duradero y huele buenísimo
3,Gomez,5.0,En el pedido son dos muestras y solo llego una...
4,Amazon Customer,2.0,Bought Chanel Perfume Samples for my Best Frie...
...,...,...,...
89,Günter S.,5.0,Super Deal!
90,Tman,4.0,Recipient was very happy with this indeed.
91,atoll,5.0,Wrong (older) chanel No 5 delivered to the (ne...
92,billersand,1.0,excellent product


In [ ]:
Review_Table.to_csv("C:/Users/이세은/Desktop/BADA/프로젝트/크롤링결과_임시저장.csv", index=False, encoding='utf-8')

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import _stop_words
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

In [ ]:
#nltk 라이브러리 영어 불용어 집합 생성
nltk_stopwords = set(stopwords.words("english"))
#sklearn 라이브러리 영어 불용어 집합 생성
sklearn_stopwords = set(_stop_words.ENGLISH_STOP_WORDS)
#사용자가 직접 추가한 불용어 목록
added_stopwords = ['channel', 'chanel', 'No 5', 'perfume', "l'eau"]


#nltk 불용어, sklearn 불용어 합친 집합 생성.
pre_my_stopwords = nltk_stopwords.union(sklearn_stopwords)
#nltk 불용어, sklearn 불용어, 사용자 정의 불용어 합친 집합 생성.
my_stopwords = pre_my_stopwords.union(added_stopwords)


#stemming: 어간 추출
ps = PorterStemmer()
#Lemmatization: 표제어 추출
lem = WordNetLemmatizer()


#단어의 품사를 결정. POS 태깅을 사용하여 단어의 품사를 추출하고, WordNet에서 사용하는 품사로 변환
def get_pos(w):  #w: 단어
    #pos_tag: nltk에서 제공하는 함수. 주어진 단어의 품사 태깅 수행.
    #튜플 형태로 품사 태깅된 결과를 반환.
    #첫 번째 요소 [0]은 단어 자체. 두 번째 요소 [1]은 해당 단어의 품사 태그.
    #[1][0]으로 품사 태그에서 첫 번째 문자를 추출 -> upper()로 대문자로 변환
    tag = pos_tag([w])[0][1][0].upper()
    if tag == "V":
        return wordnet.VERB
    elif tag == "N":
        return wordnet.NOUN
    elif tag == "J":
        return wordnet.ADJ
    elif tag == "R":
        return wordnet.ADV
    else:
        return wordnet.NOUN


#주어진 텍스트 데이터를 토큰화, 전처리
def my_tokenizer(r):
    #단어로 토큰화. 공백 및 구두점을 기준으로.
    r1 = word_tokenize(r)
    #알파벳 문자만을 남기고(구두점, 숫자 등 제거), 모두 소문자로 변환
    r2 = [w.lower() for w in r1 if w.isalpha()]
    #불용어 제거
    r3 = [w for w in r2 if not w in my_stopwords]
    #단어를 get_pos를 통해 결정된 품사에 따라 lemmatization
    r4 = [lem.lemmatize(w, get_pos(w)) for w in r3]
    return r4

In [ ]:
print(my_stopwords)

{'been', 'under', "should've", 'sometime', 'i', 'eleven', 'last', 'de', 'y', 'others', 'else', 'out', 'move', "you're", 'detail', 'when', "wouldn't", 'latter', 'because', 'just', 're', 'may', 'therein', 'about', 'nine', 'whereas', 'seeming', 'very', 'no', 'became', 'sincere', 'how', 'itself', 'is', 'himself', 'indeed', 'perhaps', 'mustn', 'bill', 'everyone', 'get', "that'll", 'together', 'we', 'whither', 'such', 'ain', 'keep', 'yet', 'before', 'back', 'herself', "haven't", 'here', 'show', 'side', 'within', 'thereby', 'except', 'don', 'having', 'that', 'on', 'among', 'con', 'since', 'few', 'had', 'ten', 'four', 'least', 'each', 'part', "l'eau", "hadn't", 'neither', 'o', 'aren', "mightn't", 'cannot', 'none', 'and', 'via', 'after', "shan't", 'isn', 'hereupon', 'so', 'often', 'these', 't', 'yours', 'the', 'behind', 'interest', 'both', 'down', 'only', 'us', 'seem', 'would', 'around', 'this', 'from', 'hereafter', 'somewhere', 'beside', 'onto', 'into', 'front', 'alone', 'an', 'amoungst', 'etc

In [ ]:
#TfidfVectorizer 객체 생성
tfidf = TfidfVectorizer(tokenizer = my_tokenizer)

In [ ]:
#Review_Table['Review'].values.tolist()로 얻은 리뷰 데이터를 TF-IDF 피처 벡터로 변환
#review_tfidf: 각 리뷰가 tf-idf 가중치를 기반으로 표현된 벡터로 저장되어 있음
review_tfidf = tfidf.fit_transform(Review_Table['Review'].values.tolist())

c:\anac\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
#tf-idf 모델에서 사용된 단어 피처 목록
tfidf_terms = tfidf.get_feature_names_out()
tfidf_terms

array(['actually', 'address', 'afford', 'afraid', 'ago', 'allone',
       'allow', 'alright', 'amaze', 'amazon', 'amber', 'amzon', 'annoy',
       'appear', 'ar', 'arrive', 'ask', 'assume', 'attempt', 'authentic',
       'away', 'awful', 'baby', 'bad', 'base', 'beautiful', 'best',
       'birthday', 'bit', 'bold', 'book', 'bottle', 'bottlewas', 'bought',
       'boy', 'britches', 'broken', 'brother', 'buenísimo', 'business',
       'buy', 'care', 'cartón', 'charge', 'cheaper', 'chiquitita',
       'christmas', 'class', 'classic', 'cleaner', 'close', 'cloy',
       'cobran', 'cologne', 'come', 'company', 'complaint',
       'congratulate', 'consigues', 'contact', 'container', 'counter',
       'crack', 'cranky', 'daughter', 'day', 'deal', 'decide', 'define',
       'definitely', 'deliver', 'delivery', 'department', 'desagradable',
       'desampárese', 'described', 'description', 'det', 'dice',
       'different', 'disappoint', 'disappointed', 'disappointment', 'do',
       'duradero', 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim import corpora

In [ ]:
#리뷰를 토큰화, 전처리한 결과를 doc_1에 저장
doc_1 = []

for d in Review:
    d1 = my_tokenizer(d)
    doc_1.append(d1)

print(doc_1)

[['glad', 'nice', 'cheaper', 'price'], ['eau', 'toilette', 'light', 'fresh', 'fragrance', 'beautiful'], ['duradero', 'huele', 'buenísimo'], ['en', 'el', 'pedido', 'son', 'do', 'muestras', 'solo', 'llego', 'una', 'e', 'muy', 'chiquitita'], ['bought', 'sample', 'best', 'friend', 'history', 'book', 'love', 'try', 'different', 'kind'], ['nice', 'purse', 'smell', 'exactly', 'like', 'mall'], ['love', 'use', 'year', 'beautiful', 'smell', 'company', 'take', 'care', 'fast', 'professional', 'manner'], ['sell', 'example', 'saysnot', 'sale', 'business', 'still', 'people', 'buy'], ['get', 'mama', 'love', 'complaint', 'whatsoever'], ['love', 'smell'], ['thank', 'well', 'thought'], ['personal', 'use'], ['glad', 'size', 'afford', 'large', 'one', 'love', 'scent'], ['come', 'timely', 'satisfied', 'purchase', 'thanks', 'free', 'gift'], ['perfect', 'amaze', 'exacly', 'say', 'negative', 'feedback'], ['item', 'quite', 'small', 'expect', 'arrive', 'good', 'timing'], ['like', 'prompt', 'delivery', 'sample', '

In [ ]:
#각 단어에 고유한 정수 ID를 할당하여 단어 집합(사전)을 만들어 줌
gensim_terms = corpora.Dictionary(doc_1)

In [ ]:
#doc_1에 있는 각 단어를 Bag of Words 형식으로 변환.
#doc_matrix: 각 문서에 대한 정보와 해당 문서에서 사용된 단어의 빈도를 나타냄
doc_matrix = [gensim_terms.doc2bow(w) for w in doc_1]

In [ ]:
doc_matrix

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)],
 [(10, 1), (11, 1), (12, 1)],
 [(13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1)],
 [(25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1)],
 [(2, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)],
 [(4, 1),
  (32, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1)],
 [(48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1)],
 [(32, 1), (56, 1), (57, 1), (58, 1), (59, 1)],
 [(32, 1), (39, 1)],
 [(60, 1), (61, 1), (62, 1)],
 [(46, 1), (63, 1)],
 [(1, 1), (32, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)],
 [(69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1)],
 [(76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1)],
 [(82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1)],
 [(33, 1), (36

In [ ]:
lda = gensim.models.ldamodel.LdaModel

In [ ]:
lda_model = lda(doc_matrix,
                num_topics = 5,  #추출하려는 토픽의 수
                id2word = gensim_terms,  #단어 집합 (사전)
                random_state = 0)

In [ ]:
#LDA 모델의 토픽들을 출력
#각 토픽에서 가장 높은 가중치를 가지는 단어들을 표시
lda_model.print_topics(num_words = 15)  #num_words: 각 토픽에서 표시할 상위 단어의 수

[(0,
  '0.016*"mom" + 0.016*"sample" + 0.016*"use" + 0.016*"price" + 0.009*"size" + 0.009*"money" + 0.009*"love" + 0.009*"smell" + 0.009*"sale" + 0.009*"order" + 0.009*"time" + 0.009*"long" + 0.009*"people" + 0.009*"en" + 0.009*"business"'),
 (1,
  '0.021*"product" + 0.019*"vendor" + 0.019*"scent" + 0.016*"like" + 0.014*"say" + 0.014*"fantastic" + 0.013*"reach" + 0.013*"seller" + 0.012*"issue" + 0.012*"money" + 0.010*"recommend" + 0.009*"toilette" + 0.009*"미디어를" + 0.009*"없습니다" + 0.009*"수"'),
 (2,
  '0.050*"like" + 0.031*"smell" + 0.024*"fragrance" + 0.023*"bottle" + 0.012*"time" + 0.011*"love" + 0.010*"long" + 0.010*"product" + 0.010*"light" + 0.010*"receive" + 0.009*"quite" + 0.009*"powder" + 0.009*"good" + 0.009*"wear" + 0.008*"year"'),
 (3,
  '0.017*"sample" + 0.017*"love" + 0.017*"vial" + 0.016*"scent" + 0.014*"smell" + 0.014*"seller" + 0.012*"return" + 0.011*"like" + 0.011*"old" + 0.010*"free" + 0.009*"molto" + 0.009*"order" + 0.009*"è" + 0.009*"thank" + 0.009*"perfect"'),
 (4,
  

In [ ]:
#각 문서에 대해 반복
for d in doc_matrix:
    #각 문서에 대해 토픽 분포를 계산. 해당 문서가 각 토픽에 속할 확률값을 출력
    print(lda_model[d])

[(0, 0.8395275), (1, 0.0400065), (2, 0.040235467), (3, 0.04022647), (4, 0.040004075)]
[(0, 0.028689496), (1, 0.028898705), (2, 0.02883565), (3, 0.028893191), (4, 0.884683)]
[(0, 0.05002007), (1, 0.050017472), (2, 0.050012268), (3, 0.7999392), (4, 0.05001093)]
[(0, 0.015448544), (1, 0.015389311), (2, 0.9380743), (3, 0.015452765), (4, 0.015635077)]
[(0, 0.018245239), (1, 0.018223334), (2, 0.018277412), (3, 0.92680234), (4, 0.018451646)]
[(0, 0.028849956), (1, 0.028619265), (2, 0.8851801), (3, 0.02865174), (4, 0.028698932)]
[(0, 0.01677616), (1, 0.01671393), (2, 0.016902171), (3, 0.01685779), (4, 0.9327499)]
[(0, 0.9109916), (1, 0.022226585), (2, 0.022247536), (3, 0.022264915), (4, 0.02226938)]
[(0, 0.033381056), (1, 0.03342968), (2, 0.03343457), (3, 0.03351949), (4, 0.8662352)]
[(0, 0.067266345), (1, 0.06678635), (2, 0.06954147), (3, 0.06813932), (4, 0.7282665)]
[(0, 0.7993134), (1, 0.050009713), (2, 0.050006878), (3, 0.05066392), (4, 0.050006077)]
[(0, 0.732008), (1, 0.0666974), (2, 0.0

In [ ]:
print(doc_matrix)

[[(0, 1), (1, 1), (2, 1), (3, 1)], [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)], [(10, 1), (11, 1), (12, 1)], [(13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)], [(25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)], [(2, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)], [(4, 1), (32, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1)], [(32, 1), (56, 1), (57, 1), (58, 1), (59, 1)], [(32, 1), (39, 1)], [(60, 1), (61, 1), (62, 1)], [(46, 1), (63, 1)], [(1, 1), (32, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)], [(69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1)], [(76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1)], [(82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1)], [(33, 1), (36, 1), (72, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1)], [(57, 1), (67, 